In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
import json
import scipy 
import torch
from torch.utils.data import Dataset, DataLoader
import logging
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (165 kB/s)
Reading package lis

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-30 03:28:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.53MB/s    in 0.2s    

2022-08-30 03:28:53 (5.53 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UA_War").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
#import data from aws s3 bucket for 1 day. This will be the starting dataframe to populate DB

from pyspark import SparkFiles
start_date = "0601"

url = f"https://databootcamps3bucket.s3.us-west-2.amazonaws.com/ua_war/UkraineWar/{start_date}_UATweets.csv.gz"
spark.sparkContext.addFile(url)

df_start_date = spark.read.option("delimiter", ",").option("encoding", "UTF-8").option("multiLine", True).option("escape", '"').csv(SparkFiles.get(f"{start_date}_UATweets.csv.gz"),  header=True, inferSchema=True)

# df.show(truncate=False) 
df_start_date_2 = df_start_date.na.fill("<empty>")

df_start_date_2.show(5)

+---+-------------------+---------------+--------------------+--------------------+---------+---------+-----------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+--------+-----------+--------------+----------+-----------------+---------------------+-----------------------+---------------------+-------------------+-----------------------+---------------+----------------+--------------------+----------------------+--------------------+
|_c0|             userid|       username|            acctdesc|            location|following|followers|totaltweets|       usercreatedts|            tweetid|      tweetcreatedts|retweetcount|                text|            hashtags|language|coordinates|favorite_count|is_retweet|original_tweet_id|original_tweet_userid|original_tweet_username|in_reply_to_status_id|in_reply_to_user_id|in_reply_to_screen_name|is_quote_status|quoted_status_id|quoted_status_userid|quoted_status_username|      

In [ ]:
#see what the datatypes for df_8018 
df_start_date.dtypes

[('_c0', 'int'),
 ('userid', 'bigint'),
 ('username', 'string'),
 ('acctdesc', 'string'),
 ('location', 'string'),
 ('following', 'int'),
 ('followers', 'int'),
 ('totaltweets', 'int'),
 ('usercreatedts', 'string'),
 ('tweetid', 'bigint'),
 ('tweetcreatedts', 'string'),
 ('retweetcount', 'int'),
 ('text', 'string'),
 ('hashtags', 'string'),
 ('language', 'string'),
 ('coordinates', 'string'),
 ('favorite_count', 'int'),
 ('is_retweet', 'boolean'),
 ('original_tweet_id', 'bigint'),
 ('original_tweet_userid', 'bigint'),
 ('original_tweet_username', 'string'),
 ('in_reply_to_status_id', 'bigint'),
 ('in_reply_to_user_id', 'bigint'),
 ('in_reply_to_screen_name', 'string'),
 ('is_quote_status', 'boolean'),
 ('quoted_status_id', 'bigint'),
 ('quoted_status_userid', 'bigint'),
 ('quoted_status_username', 'string'),
 ('extractedts', 'string')]

In [ ]:
#use this to determine which type of drop is needed. Also not all all dates have same number of columns 
len(df_start_date.columns)

29

In [ ]:
from pyspark import SparkFiles
from pyspark.sql.functions import *

#use this to determine which type of drop is needed. Also not all all dates have same number of columns 
df_start_date_count = len(df_start_date.columns)
if df_start_date_count == 18:
  df_start_date_2 = df_start_date_2.withColumn("is_retweet", lit(None).cast('boolean')) 
  df_start_date_2 = df_start_date_2.withColumn("is_quote_status", lit(None).cast('boolean')) 

len(df_start_date_2.columns)

29

In [ ]:
from pyspark.sql.functions import *

#1. Drop Uncessary Columns using the data set as a starting point. 
cleaned_df = df_start_date_2.drop("userid", "_c0", "acctdesc", "location", "tweetid", "coordinates", "original_tweet_id", "original_tweet_userid", "original_tweet_username", "in_reply_to_status_id", "in_reply_to_screen_name", 'in_reply_to_user_id', "quoted_status_id", "quoted_status_username", "quoted_status_userid")

#2. change columns from string to year:month:day date format
cleaned_df = cleaned_df.withColumn("usercreatedts",to_date("usercreatedts"))
cleaned_df = cleaned_df.withColumn("tweetcreatedts",to_date("tweetcreatedts"))
cleaned_df = cleaned_df.withColumn("extractedts",to_date("extractedts"))

#3. filter out langauge = english only
cleaned_df = cleaned_df.filter(cleaned_df["language"]=="en")
cleaned_df = cleaned_df.filter(cleaned_df["usercreatedts"] < "2009-01-01")

cleaned_df.show(5)

+---------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+
| username|following|followers|totaltweets|usercreatedts|tweetcreatedts|retweetcount|                text|            hashtags|language|favorite_count|is_retweet|is_quote_status|extractedts|
+---------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+
|PSUdotcom|     1322|    46599|      81896|   2008-11-06|    2022-06-01|           2|Assassins Creed O...|[{'text': 'Assass...|      en|            11|     false|          false| 2022-06-01|
|  hk_kure|      544|       72|       4044|   2008-07-25|    2022-06-01|        2540|Russian soldiers ...|[{'text': 'Ukrain...|      en|             0|      true|          false| 2022-06-01|
|  DawgOnU|     1032|      247|      15636|  

In [ ]:
cleaned_df_count = cleaned_df.count()
cleaned_df_count

3585

In [ ]:
date_arr_june = [ "0602", "0603", "0604", "0605", "0606_to_08", "0609","0610", "0611", "0612", "0613", "0614", "0615", "0616", "0617", "0618", "0619", "0620", "0621", "0622", "0623", "0624", "0625", "0626", "0627", "0628", "0629", "0630"]

In [ ]:
from pyspark import SparkFiles
from pyspark.sql.functions import *

url = "https://databootcamps3bucket.s3.us-west-2.amazonaws.com/ua_war/UkraineWar"

for day in date_arr_june:
      #load the data from aws
      aws_url = f"{url}/{day}_UATweets.csv.gz"
      spark.sparkContext.addFile(aws_url)
      temp_df = spark.read.option("delimiter", ",").option("encoding", "UTF-8").option("multiLine", True).option("escape", '"').csv(SparkFiles.get(f"{day}_UATweets.csv.gz"),  header=True, inferSchema=True)
      
      #keep count number of columns to determine which if else block it will hit
      temp_count = len(temp_df.columns)

      #change columns from string to year:month:day date format
      temp_df = temp_df.withColumn("usercreatedts",to_date("usercreatedts"))
      temp_df = temp_df.withColumn("tweetcreatedts",to_date("tweetcreatedts"))
      temp_df = temp_df.withColumn("extractedts",to_date("extractedts"))

      #filter out data for english only 
      temp_df = temp_df.filter(temp_df["language"]=="en")
      #filter out usercreated after 2009 
      temp_df = temp_df.filter(temp_df["usercreatedts"] < "2009-01-01")

      #fill in null values 
      temp_df_2 = temp_df.na.fill("<empty>")

      #some days the data columns has less columns then other days 
      if temp_count == 18:
        temp_df_3 = temp_df_2.drop("userid", "_c0", "acctdesc", "location", "tweetid", "coordinates")
        temp_df_3 = temp_df_3.withColumn("is_retweet", lit(None).cast('boolean')) 
        temp_df_3 = temp_df_3.withColumn("is_quote_status", lit(None).cast('boolean')) 

      elif temp_count == 29:
        temp_df_3 = temp_df_2.drop("userid", "_c0", "acctdesc", "location", "tweetid", "coordinates", "original_tweet_id", "original_tweet_userid", "original_tweet_username", "in_reply_to_status_id", "in_reply_to_screen_name", 'in_reply_to_user_id', "quoted_status_id", "quoted_status_username", "quoted_status_userid")

      else:
        print(f"Error on {day}_UATweets.csv.gz and column count {temp_count}")

      cleaned_df = cleaned_df.unionByName(temp_df_3)
      print(f"{day} - {cleaned_df.count()}")


0602 - 6762
0603 - 10311
0604 - 13259
0605 - 16394
0606_to_08 - 26390
0609 - 29660
0610 - 33222
0611 - 36476
0612 - 39498
0613 - 42753
0614 - 46103
0615 - 49288
0616 - 52943
0617 - 56195
0618 - 58793
0619 - 61045
0620 - 63627
0621 - 66146
0622 - 69157
0623 - 72385
0624 - 75056
0625 - 77238
0626 - 80070
0627 - 83200
0628 - 86502
0629 - 89279
0630 - 91817


In [ ]:
cleaned_df.show(5)

+---------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+
| username|following|followers|totaltweets|usercreatedts|tweetcreatedts|retweetcount|                text|            hashtags|language|favorite_count|is_retweet|is_quote_status|extractedts|
+---------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+
|PSUdotcom|     1322|    46599|      81896|   2008-11-06|    2022-06-01|           2|Assassins Creed O...|[{'text': 'Assass...|      en|            11|     false|          false| 2022-06-01|
|  hk_kure|      544|       72|       4044|   2008-07-25|    2022-06-01|        2540|Russian soldiers ...|[{'text': 'Ukrain...|      en|             0|      true|          false| 2022-06-01|
|  DawgOnU|     1032|      247|      15636|  

In [ ]:
#add week column for "tweetcreatedts"
cleaned_df = cleaned_df.withColumn("weekofyear",weekofyear("tweetcreatedts"))

In [ ]:
cleaned_df.show(5)

+---------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+----------+
| username|following|followers|totaltweets|usercreatedts|tweetcreatedts|retweetcount|                text|            hashtags|language|favorite_count|is_retweet|is_quote_status|extractedts|weekofyear|
+---------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+----------+
|PSUdotcom|     1322|    46599|      81896|   2008-11-06|    2022-06-01|           2|Assassins Creed O...|[{'text': 'Assass...|      en|            11|     false|          false| 2022-06-01|        22|
|  hk_kure|      544|       72|       4044|   2008-07-25|    2022-06-01|        2540|Russian soldiers ...|[{'text': 'Ukrain...|      en|             0|      true|          false| 2022-06-01|  

In [ ]:
cleaned_df.dtypes

[('username', 'string'),
 ('following', 'int'),
 ('followers', 'int'),
 ('totaltweets', 'int'),
 ('usercreatedts', 'date'),
 ('tweetcreatedts', 'date'),
 ('retweetcount', 'int'),
 ('text', 'string'),
 ('hashtags', 'string'),
 ('language', 'string'),
 ('favorite_count', 'int'),
 ('is_retweet', 'boolean'),
 ('is_quote_status', 'boolean'),
 ('extractedts', 'date'),
 ('weekofyear', 'int')]

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com:5432/ua_data"
config = {"user":"uatweets", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
cleaned_df.write.jdbc(url=jdbc_url, table='tweets_table', mode=mode, properties=config)